In [1]:
import csv
import numpy as np
import ast
import json
import os
import sys
import re
from textblob import TextBlob
import collections
import pandas as pd
import tldextract
import requests
from urllib.parse import urlparse, urlsplit, urlunsplit
from time import sleep

In [2]:
def featuresExtraction(features_file):
    convertedArray = json.load(open(features_file, 'r'))
    features = []
    for i, j in enumerate(convertedArray):
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            features.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            features.append(j['extended_tweet']['full_text'])
        else:
            features.append(j['text'])
    
    return np.array(features)

In [3]:
def labelExtraction(label_file):
#     csv_file = open(file)
#     csv_reader = csv.reader(csv_file, delimiter = ' ')
#     labels = []
#     for row in csv_reader:
#         labels.append(row[0])
    labels = np.genfromtxt(label_file,dtype=None)
    return labels

In [4]:
def reduction(array, key):
    reducedArray = []
    for i, j in enumerate(array):
        if len(reducedArray) == 0:
            reducedArray.append(j)
        else:
            count = 0
            for k in range(len(reducedArray)):
                if a[k] != b[i]:
                    count = count + 1
            
                if count == len(a):
                    a.append(b[i])
    
    for i, j in enumerate(convertedArray):
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            features.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            features.append(j['extended_tweet']['full_text'])
        else:
            features.append(j['text'])
    return reducedArray

In [5]:
def getCredentials():
    app_key = "jGVy1ilV55A7P24eJHuousM1b"
    app_secret = "wrPOxGirrXEUQsU7Pk0jqaqt7nLYq10TmR0E6QdZPSlvvrktE8"
    oauth_token = "1036998187880460289-K5YKVDBIlK5e1bsgsxzvmGk2z2BzTO"
    oauth_token_secret = "ogxeyO9mgweKrFEOZhn638EvTVvZjn06ynVwviShsVOzZ"
    
    return app_key, app_secret, oauth_token, oauth_token_secret

In [6]:
def featuresExtraction2(features_file):
    convertedArray = json.load(open(features_file, 'r'))
    final = []
    textFeatures = []
    
    for i, j in enumerate(convertedArray):
        tempDict = {}
        tempDict['id'] = j['id']
        tempDict['date'] = j['created_at']
        tempDict['username'] = j['user']['screen_name']
        tempDict['hashtags'] = j['entities']['hashtags']
        tempDict['user_mentions'] = j['entities']['user_mentions']
        tempDict['favorites'] = j['favorite_count']
        
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            textFeatures.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            textFeatures.append(j['extended_tweet']['full_text'])
        else:
            textFeatures.append(j['text'])
        
        final.append(tempDict)
    
    return final, textFeatures

In [7]:
def authorization(key):
    if key == 'e' or 'E':
        app_key = "jGVy1ilV55A7P24eJHuousM1b"
        app_secret = "wrPOxGirrXEUQsU7Pk0jqaqt7nLYq10TmR0E6QdZPSlvvrktE8"
        oauth_token = "1036998187880460289-K5YKVDBIlK5e1bsgsxzvmGk2z2BzTO"
        oauth_token_secret = "ogxeyO9mgweKrFEOZhn638EvTVvZjn06ynVwviShsVOzZ"
        return app_key, app_secret, oauth_token, oauth_token_secret
    elif key == 'v' or 'V':
        app_key = "d"
    else:
        print("Wrong identification key")

In [8]:
def cred_Processing(tweetsFile):
    idRegEx = re.compile(r".*ID=")
    endElRegEX = re.compile(r"'.*")
    output = open("outputTweets.json", "w")
    
    with open(tweetsFile, "r") as f:
        header = f.readline()
        
        for line in f:
            topicData = line.split("\t")
            
            topicKey = topicData[0]
            topicTerms = topicData[1]
            topicTweetCount = topicData[2]
            tweetIdList = topicData[3]
            
            print(topicKey)
            
            realTweetsIds = []
            
            idElements = tweetIdList.split("),")
            for element in idElements:
                elArr = element.split(",")
                idEl = list(filter(lambda x: "ID" in x, elArr))[0]
                idEl = idRegEx.sub("", idEl)
                idEl = endElRegEX.sub("", idEl)
                
                realTweetsIds.append(int(idEl))
                
            realTweetsIds = list(set(realTweetsIds))
            
            topicMap = {
                "key" : topicKey,
                "terms" : topicTerms.split(","),
                "count" : topicTweetCount, 
                "tweets" : realTweetsIds
            }
            
            json.dump(topicMap, output, sort_keys=True)
            output.write("\n")
    output.close()

In [9]:
def rating_processing(ratingFile, tweetsFile):
    tweetsMap = {}
    
    with open(tweetsFile, "r") as f:
        for line in f:
            tweetData = json.loads(line)
            tweetsMap[tweetData["key"]] = tweetData
            
    output = open(r"C:\Users\vdoan\Documents\College\SummerResearch\SmResearch\Data\CREDBANK\finalOutput.json", "w")
    
    with open(ratingFile, "r") as f:
        header = f.readline()
        
        for line in f:
            topicData = line.split("\t")
            topicKey = topicData[0]
            topicTerms = topicData[1]
            ratings = topicData[2]
            reasons = topicData[3]
            
            try:
                ratings = list(map(lambda x: int(x.strip().replace("'", "")), ratings.replace("[", "").replace("]", "").split(",")))
                ratings = np.array(ratings)
                tweetsMap[topicKey]["mean"] = np.mean(ratings)
                print("Inserting ratings for {}".format(topicKey))
                tweetsMap[topicKey]["ratings"] = ratings.tolist()
                print("Success")
                print("Inserting mean for {}".format(topicKey))
                tweetsMap[topicKey]["mean"] = np.mean(ratings)
                print("Success")
                topicMap = tweetsMap[topicKey]
                print(topicMap["key"], topicMap["mean"])
                json.dump(topicMap, output, sort_keys=True)
                output.write("\n")
            except:
                print("{} is missing.".format(topicKey))
            
            
            
    output.close()

In [10]:
def binaryConversion(outputFile):
    labels = {}
    with open(outputFile, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            if(tweet['mean'] >= 1.9):
                labels[tweet['key']] = 1
            elif(tweet['mean'] <= 1.467):
                labels[tweet['key']] = 0
    return labels

In [11]:
#TODO: Retweet and date
def singleMessageFeatureExtraction(tweet_json):
    #print(tweet_json)
    retweets = 0
    try:
        retweets = tweet_json["tweet"]["retweet_status"]
        retweets = 1
    except:
        retweets = 0
    date = tweet_json["tweet"]["created_at"].split(" ")[0]
    dates = []
    if date == "Mon":
        dates = [1,0,0,0,0,0,0]
    elif date == "Tue":
        dates = [0,1,0,0,0,0,0]
    elif date == "Wed":
        dates = [0,0,1,0,0,0,0]
    elif date == "Thu":
        dates = [0,0,0,1,0,0,0]
    elif date == "Fri":
        dates = [0,0,0,0,1,0,0]
    elif date == "Sat":
        dates = [0,0,0,0,0,1,0]
    elif date == "Sun":
        dates = [0,0,0,0,0,0,1]
    return retweets, dates

In [12]:
def unshorten_url(url):
    return requests.head(url, allow_redirects=True).url

In [13]:
def messageFeatureExtraction(tweets, tweet_json, threshold):
    tweetsFeatures = None
    np.array(tweetsFeatures)
    count = 0
    for tweet in tweets:
        print("Starting feature extraction for tweet {} of {}".format(count, len(tweets)))
        indiTweetFeature = []
        #Length of characters 1
        print("Adding Length of charactrs")
        indiTweetFeature.append(len(tweet))
        #print(indiTweetFeature)
        #print(indiTweetFeature)
        
        print("Adding words")
        #Length of words 2
        indiTweetFeature.append(len(tweet.split(" ")))
        #print(indiTweetFeature)
        #print(indiTweetFeature)
        
        print("Adding question mark")
        #Contains question mark 3
        results = collections.Counter(tweet)
        if(results['?'] > 1):
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)
        #print(indiTweetFeature)

        print('Adding exclamation mark')
        #Contains exclamation 4
        if(results['!'] > 1):
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)
        #print(indiTweetFeature)

        print("Adding smiles")
        #Contains emoticon smile 5
        emojis = "😊"
        indiTweetFeature.append(tweet.count(emojis))
        #print(indiTweetFeature)
        #print(indiTweetFeature)

        print("!?")
        #Contains multiple question or exclamation marks 6
        if results['?'] and results['!'] > 1:
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)
        #print(indiTweetFeature)

        print("Adding first person pronouns")
        #Contains pronoun first, sceond, third 7
        first_pronouns = ["i", "we"]
        tweetSplit = tweet.split(" ")
        for x in range(len(tweetSplit)):
            if(tweetSplit[x].lower() in first_pronouns):
                indiTweetFeature.append(1)
                break;
            if(x == len(tweetSplit)-1):
                indiTweetFeature.append(0)
        #print(indiTweetFeature)
        #print(indiTweetFeature)
        print("Adding second person pronouns")
        second_pronouns = ["you"]
        tweetSplit = tweet.split(" ")
        for x in range(len(tweetSplit)):
            if(tweetSplit[x].lower() in second_pronouns):
                indiTweetFeature.append(1)
                break;
            if(x == len(tweetSplit)-1):
                indiTweetFeature.append(0)
        #print(indiTweetFeature)
        #print(indiTweetFeature)
        print("Adding third person pronouns")
        third_pronouns = ["he", "him," "she", "her", "they", "them", "it"]

        tweetSplit = tweet.split(" ")
        for x in range(len(tweetSplit)):
            if(tweetSplit[x].lower() in third_pronouns):
                indiTweetFeature.append(1)
                break;
            if(x == len(tweetSplit)-1):
                indiTweetFeature.append(0)
        #print(indiTweetFeature)
       #print(indiTweetFeature)   
        
        print("Adding uppercase")
        #Count uppercase letters 8
        upperLetterCount = 0
        for letter in tweet:
            if(letter.isupper()):
                upperLetterCount += 1
        indiTweetFeature.append(upperLetterCount)
        #print(indiTweetFeature)
        #print(indiTweetFeature)

        print("Adding url")
        #Number of URLs 9
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet) 
        indiTweetFeature.append(len(url))
        #print(indiTweetFeature)
        #print(indiTweetFeature)
        
        #Contains popular domain top 100,1000,10000
        print("Adding popular domain")
        file = pd.read_csv('../../Data/top-1m.csv')
        try:
            urls = tweet_json[count]['tweet']['entities']['urls']
        except:
            print(tweet_json[count]['tweet'])
        top100 = 0
        top1000 = 0
        top10000 = 0

        for url in urls:
            urlParsed = None
            rank = None
            try:
                split_url = urlsplit(url['expanded_url'])   
                shortUrl = ''
                if(split_url.netloc[:4] == 'www.'):
                    shortUrl = split_url.netloc[4:]
                else:
                    shortUrl = split_url.netloc
                print(shortUrl)
                print(file.loc[file['site'] == shortUrl]['rank'])
                rank = int(file.loc[file['site'] == split_url.netloc]['rank'])
            except Exception as e:
                rank = 10001
            if rank <= 100:
                top100 = 1
            if rank <= 1000:
                top1000 = 1
            if rank <= 10000:
                top10000 = 1
        
        indiTweetFeature.append(top100)
        indiTweetFeature.append(top1000)
        indiTweetFeature.append(top10000)
                
        print("Adding mention")
        #Contains user mention 10
        result = re.findall("(^|[^@\w])@(\w{1,15})", tweet)
        if(len(result) >= threshold):
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)
        #print(indiTweetFeature)
    
        # Length of hashtags 11
        print("Adding hashtags")
        hashtags = [i  for i in tweet.split() if i.startswith("#") ]
        indiTweetFeature.append(len(hashtags))
        #print(indiTweetFeature)
     
        #Contains stock symbol 12
        print("Addings stocks")
        file = open('../../Data/nasdaqlisted.txt', 'r')
        filecontents = file.readlines()
        stock_symbols = []
        for line in filecontents:
            splitline = line.split("|")
            stock_symbols.append(splitline[0])
            
        del stock_symbols[0]
        
        stockCount = 0
        for word in tweetSplit:
            if(word in stock_symbols):
                stockCount += 1
        
        if(stockCount > 0):
            indiTweetFeature.append(1)
        else:
            indiTweetFeature.append(0)
        #print(indiTweetFeature)
                
        retweets, date = singleMessageFeatureExtraction(tweet_json[count])
        print("Adding retweets")
        indiTweetFeature.append(retweets)
        #print(indiTweetFeature)
        print("Adding date")
        indiTweetFeature = indiTweetFeature + date
        #print(indiTweetFeature)
        
        print("Adding sentiment")
        #Sentiment 13, 14, 15
        positiveWords = 0
        negativeWords= 0
    
        tweetSentiment = TextBlob(tweet)
        sentimentScore = tweetSentiment.sentiment[0]
    
        for word in tweetSentiment:
            word = TextBlob(word)
            if(word.sentiment.polarity) > 0:
                positiveWords = positiveWords + 1
            elif(word.sentiment.polarity) < 0:
                negativeWords = negativeWords + 1
        indiTweetFeature.append(sentimentScore)
        #print(indiTweetFeature)
        
        #print(indiTweetFeature)
        indiTweetFeature = [indiTweetFeature]
        if(count == 0):
            #print("hello")
            tweetsFeatures = indiTweetFeature
            #print(tweetsFeatures.shape)
            print("Length of array: {}".format(len(tweetsFeatures[count])))
        else:
            tweetsFeatures = np.append(tweetsFeatures, indiTweetFeature, axis = 0)
            print(tweetsFeatures.shape)
        #print(len(indiTweetFeature))
        print("Finished feature extraction for tweet {} of {}".format(count, len(tweets)))
        count += 1
        
#         if count == 3:
#             break;
    return tweetsFeatures

def topicFeatureExtraction():
    

In [ ]:
def topicFeatureExtraction(topics, threshold):
    
    all_features = None
    np.array(all_features)
    
    first_pronouns = set(["i", "we"])
    second_pronouns = set(["you"])
    third_pronouns = set(["he", "him," "she", "her", "they", "them", "it"])
    
    for topic in topics: 
        print("Starting topic feature extraction")
        print("Current topic: ", topic)
        current_topic = topics[topic]
        
        
        #prop for proportion, fr for frequency 
        int total_no_tweets = len(current_topic)
        
        int fr_quest_mark = 0
        int fr_excl_mark = 0
        
        int fr_emot_smile = 0 
    
        int fr_tweets_hashtag = 0
        
        
        
        int fr_pron_first = 0
        int fr_pron_sec = 0
        int fr_pron_third = 0
        
        #Polarity = Average sentiment score of tweets 
        int avg_polarity = 0 
        int disagreement = 0
        
        for tweet in current_topic: 
            
            split_tweet = split(tweet)
            split_tweet_word = set(tweet.split(" "))

            if '?' in split_tweet:
                fr_quest_mark += 1
            if '!' in split_tweet:
                fr_excl_mark += 1
            if '😊' in split_tweet:
                fr_emot_smile += 1
            if '#' in split_tweet:
                fr_tweets_hashtag += 1
            
            
            if (first_pronounces & split_tweet_word):
                fr_pron_first += 1
            if (second_pronounces & split_tweet_word):
                fr_pron_sec += 1
            if (third_pronounces & split_tweet_word):
                fr_pron_third += 1 
                
            
            tweetSentiment = TextBlob(tweet)
            sentimentScore = tweetSentiment.sentiment[0]
            disagreementScore = tweetSentiment.sentiment[1]
            avg_polarity += sentimentScore
            disagreement += disagreementScore
                
                
                
        int prop_quest_mark = fr_quest_mark / total_no_tweets
        int prop_excl_mark = fr_excl_mark / total_no_tweets
        int prop_emot_smile = fr_emot_smile / total_no_tweets
        int prop_tweets_hashtag = fr_tweets_hashtag / total_no_tweets

        
        int prop_pron_first = fr_pron_first / total_no_tweets
        int prop_pron_sec = fr_pron_sec / total_no_tweets
        int prop_pron_third = fr_pron_third / total_no_tweets
        
        avg_polarity = avg_polarity / total_no_tweets
        
           
        int disagreement = 0
    
        all_features.append(avg_polarity)
        all_features.append(objectivity)
        all_features.append(disagreement)
        all_features.append(fr_quest_mark)
        all_features.append(prop_quest_mark)
        all_features.append(fr_excl_mark)
        all_features.append(prop_excl_mark)
        all_features.append(fr_emot_smile)
        all_features.append(prop_emot_smile)
        all_features.append(fr_pron_first)
        all_features.append(prop_pron_first)
        all_features.append(fr_pron_sec)
        all_features.append(prop_pron_sec)
        all_features.append(fr_pron_third)
        all_features.append(prop_pron_third)

    return all_features 
    

In [ ]:
import datetime
from datetime import date

In [ ]:
def userFeatureExtraction(topics, threshold):
    all_features = None
    np.array(all_features)
    
    #Average account age format : "created_at": "Fri Jun 26 07:47:06 +0000 2009"


    #When an account was created and the relevant tweet was authored
    
    
    #Network Density

    for topic in topics:
        #Avg date create
        int avg_date_user_create = 0
        
        #Average follower count
        int avg_follower_cnt = 0
    
    
        #Average friend count
        int avg_friend_cnt = 0 
    
        #Average authored status count 
        int avg_aut_stat_cnt = 0
    
        #Frequency of verified author 
        int fr_veri_aut = 0 
    
        #Author of the First Tweet is Verified
        int fr_first_veri = 0
        current_topic = topics[topic]
        no_tweet = len(topic)
        
        int avg_date_tweet_create = 0
        
        #TODO: how to format this?
        for tweet in current_topic:
            
            #Average account age format : "created_at": "Fri Jun 26 07:47:06 +0000 2009"
            user_created_string = current_topic['user']['created_at']
            tweet_created_string = current_topic['created_at']
            user_created_string_list = user_created_string.split(" ")
            tweet_created_string_list = tweet_created_string.split(" ")
            int day_user = user_created_string_list[2]
            int month_user = 0
            int year_user = user_created_string_list[5]
            
            int day_tweet = tweet_created_string_list[2]
            int month_tweet = 0
            int year_tweet = tweet_created_string_list[5]

            if (user_created_string_list[1] == 'Jan'):
                month = 1
            elif (user_created_string_list[1] == 'Feb'):
                month = 2
            elif (user_created_string_list[1] == 'Mar'):
                month = 3
            elif (user_created_string_list[1] == 'Apr'):
                month = 4
            elif (user_created_string_list[1] == 'May'):
                month = 5
            elif (user_created_string_list[1] == 'Jun'):
                month = 6
            elif (user_created_string_list[1] == 'Jul'):
                month = 7
            elif (user_created_string_list[1] == 'Aug'):
                month = 8
            elif (user_created_string_list[1] == 'Sep'):
                month = 9
            elif (user_created_string_list[1] == 'Oct'):
                month = 10
            elif (user_created_string_list[1] == 'Nov'):
                month = 11
            elif (user_created_string_list[1] == 'Dec'):
                month = 12
                
                
            if (tweet_created_string_list[1] == 'Jan'):
                month = 1
            elif (tweet_created_string_list[1] == 'Feb'):
                month = 2
            elif (tweet_created_string_list[1] == 'Mar'):
                month = 3
            elif (tweet_created_string_list[1] == 'Apr'):
                month = 4
            elif (tweet_created_string_list[1] == 'May'):
                month = 5
            elif (tweet_created_string_list[1] == 'Jun'):
                month = 6
            elif (tweet_created_string_list[1] == 'Jul'):
                month = 7
            elif (tweet_created_string_list[1] == 'Aug'):
                month = 8
            elif (tweet_created_string_list[1] == 'Sep'):
                month = 9
            elif (tweet_created_string_list[1] == 'Oct'):
                month = 10
            elif (tweet_created_string_list[1] == 'Nov'):
                month = 11
            elif (tweet_created_string_list[1] == 'Dec'):
                month = 12
            
            user_created_date = date(year_user,month_user,day_user)
            tweet_created_date = date(year_tweet,month_tweet,day_tweet)
            current_date = datetime.datetime.today()
            user_delta = current_date - user_created_date
            tweet_delta = current_date - tweet_created_date
            avg_date_user_create += user_delta.days
            avg_date_tweet_create += tweet_delta.days
            
            
            
            int follower_count = current_topic['user']['followers_count']
        
            int friend_count = current_topic['user']['friends_count']
        
            int aut_stat_count = current_topic['user']['statuses_count']
        
            #0 for false, 1 for true
            int is_verified = 0
        
            if (current_topic['user']['verified'] == 'true'):
                is_verified = 1
            else:
                is_verified = 0
            
            if (current_topic['in_reply_to_status_id'] != 'null') && (current_topic['user']['verified'] == 'true':
                fr_first_veri += 1
            
                                                                      
            avg_follower_cnt += follower_count
            avg_friend_cnt += friend_count
            avg_aut_stat_cnt += aut_stat_count
            fr_veri_aut += is_verified                                                              
        avg_date_user_create = avg_date_user_create / no_tweet                                                              
        avg_follower_cnt = avg_follower_cnt / no_tweet
        avg_friend_cnt = avg_friend_cnt / no_tweet
        avg_aut_stat_cnt = avg_aut_stat_cnt / no_tweet
        avg_date_tweet_create = avg_date_tweet_create / no_tweet                                                    
        
        current_topic_feature = None
        np.array(current_topic_feature)
        
        current_topic_feature.append(avg_date_user_create)                                                              
        current_topic_feature.append(avg_follower_cnt)
        current_topic_feature.append(avg_friend_cnt)
        current_topic_feature.append(avg_aut_stat_cnt)
        current_topic_feature.append(fr_veri_aut)
        current_topic_feature.append(fr_first_veri)
        current_topic_feature.append()                                                              
                                                                      
        all_features.append(current_topic_feature)                                                              

           
        
    return all_features 
    
    

def propagationFeatureExtraction():
    